# Pharmacokinetics and Pharmacodynamics of Myelotoxicity

### Contents
- [Introduction](introduction.ipynb)
- **Exploring Data**
    - [Overview](#Overview)
    - [PK Data](#PK-Data)
    - [PD Data](#PD-Data)
- [PK model - Naive Pooled Inference](PK_naive_pooled_inference.ipynb)
- [PD friberg model - Naive Pooled Inference](PD_naive_pooled_inference.ipynb)

## Exploring Data

In [55]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas

### Overview

In [56]:
df = pandas.read_csv("0470-2008_2018-05-09.csv")
drug_names = df.DRUG.unique()
drug_names = drug_names[drug_names != 'Controls']
print("Drugs in dataset:", drug_names)

Drugs in dataset: ['Vinorelbine' 'Docetaxel' 'Irinotecan' 'Topotecan' 'Vinflunine']


In [57]:
# Select drug:
drug = 'Docetaxel'

In [58]:
df_drug = df.loc[(df['DRUG'] == drug) | (df['DRUG'] == 'Controls')]
observation_names = df_drug.YNAME.unique()
print("Observations in dataset:", observation_names)

Observations in dataset: ['Docetaxel' 'Red blood cells' 'Hemoglobin' 'Platelets '
 'White blood cells' 'Neutrophiles absolute' 'Lymphocytes absolute'
 'Monocytes absolute' 'Eosinophils absolute' 'Basophils absolute']


In [59]:
# Select Obsevation(s):
observations = ["Red blood cells", "White blood cells"]

In [61]:
# Create plot
for name in observations:
    df_obs = df_drug.loc[df['YNAME'] == name]
    df_obs= df_obs.drop(df_obs[df_obs['OBS'] == '.'].index)
    df_obs['OBS'] = df_obs['OBS'].astype(float)
    x_label = "Time, hours"
    y_label = name + ", " + df_obs['UNIT'].values[0]
    if name == drug:
        tick_distance = 1
    else:
        tick_distance = 48
    fig = px.scatter(
        df_obs,
        title=drug + " Data: " + name,
        x="TIME",
        y="OBS",
        # line_group="ID",
        color="DOSE",
        size="WT",
        size_max=10,
        width=800, 
        height=500,
        hover_data=list(df.columns))
    fig.update_xaxes(title_text=x_label, tick0=0, dtick=tick_distance)
    fig.update_yaxes(title_text=y_label)
    # fig.update_traces(mode='markers+lines')
    fig.show()

## PK Data

Lets look at the data for determining the Pharmacokinetic behaviour to see what it can tell us.

In [7]:
df = pandas.read_csv("0470-2008_2018-05-09.csv")
drug_names = df.DRUG.unique()
df = df.sort_values(['DOSE', 'TIME'], ascending=True)
drug_names = drug_names[drug_names != 'Controls']

In [8]:
for drug in drug_names:
    df_drug = df.loc[((df['DRUG'] == drug) | (df['DRUG'] == 'Controls'))&(df['YNAME'] == drug)]
    df_drug = df_drug.drop(df_drug[df_drug['OBS'] == '.'].index)

    df_drug = df_drug.astype({'OBS': 'float64', 'DOSE': 'float64'})
    df_drug['Dose-Normalised'] = df_drug['OBS']/df_drug['DOSE']

    x_label = "Time, hours"
    y_label = drug + ", " + df_drug['UNIT'].values[0]
    dose_categories = df_drug.DOSE.unique()

    fig = px.line(
        df_drug,
        title=drug + " Concentrations",
        x="TIME",
        y="OBS",
        line_group="ID",
        color="DOSE",
        width=800, 
        height=500,
        # category_orders = {"Dose":["0", "5", "10", "15"]},
        hover_data=list(df.columns))

    fig.update_xaxes(title_text=x_label)
    fig.update_yaxes(title_text=y_label)
    fig.update_layout(yaxis_type="log")
    fig.update_traces(mode='markers+lines')
    fig.show()

    # for dose in dose_categories:
    dose_unit = "unit" # df_drug.loc[(df_stats['DOSE'] == dose)]['UNIT'].values[0]
    df_stats = df_drug[['TIME', 'DOSE', 'Dose-Normalised']]
    # df_stats = df_stats.astype({'OBS': 'float64'})
    df_stats = df_stats.groupby(["TIME", "DOSE"], as_index=False).filter(lambda x: len(x) > 1).groupby(["TIME", "DOSE"], as_index=False).agg({'Dose-Normalised':['mean','std']})
    df_stats.columns = ['TIME', "DOSE", 'mean', 'std']
    y_label = "Dose Normalised "+ drug + " Level (Observation/Dose)"

    fig = px.line(
        df_stats,
        title=drug + " Concentation Mean",
        x="TIME",
        y="mean",
        # error_y = "std",
        # facet_col="DOSE",
        color="DOSE",
        width=800, 
        height=500,
    )

    fig.update_xaxes(title_text=x_label)
    fig.update_yaxes(title_text=y_label)
    fig.update_layout(yaxis_type="log")
    fig.update_traces(mode='markers+lines')
    fig.show()

## PD Data

In [50]:
df = pandas.read_csv("0470-2008_2018-05-09.csv")
drug_names = df.DRUG.unique()
observation_name = "Red blood cells"
df = df.sort_values(['DOSE', 'TIME'], ascending=True)

In [51]:
# Finding Random participants to graph

# Relevant Groups and their IDs
ids = {
    'Test_Group_1_ids' : np.arange(5061, 5073), # Control Ids 5061-5072
    'Test_Group_2_ids' : np.arange(5073, 5085), # Vinorelbine 5 Ids 5073-5084
    'Test_Group_3_ids' : np.arange(5085, 5097), # Vinorelbine 10 5085-5096
    'Test_Group_4_ids' : np.arange(5121, 5133), # Docetaxel 5 5121-5132
    'Test_Group_5_ids' : np.arange(5133, 5145), # Docetaxel 10 5133-5144
    'Test_Group_6_ids' : np.arange(5145, 5157), # Irinotecan 34 5145-5156
    'Test_Group_7_ids' : np.concatenate((np.array([5158,5159]), np.arange(5160, 5168))), # Irinotecan 68 5158-5159, 5160-5168
    'Test_Group_1b_ids' : np.arange(5097, 5109), # Vinorelbine 20 5097-5108
    'Test_Group_2b_ids' : np.arange(5109, 5121), # Docetaxel15 Ids 5109-5120
    'Test_Group_3b_ids' : np.arange(5188, 5200), # Vinoflunine 10 5188-5199
    'Test_Group_4b_ids' : np.arange(5200, 5212), # Vinoflunine 20 5200-5211
    'Test_Group_9b_ids' : np.arange(5176, 5188), # Topotecan 7.5 5176-5187
    'Test_Group_10b_ids' : np.arange(5794, 5806), # Topotecan 15 5794-5805
}
#Selecting randomly from each group
num_to_select = 3
samples = np.array([])
for group in ids:
    sample_from_group = np.random.choice(ids[group], size=num_to_select, replace=False)
    samples = np.concatenate((samples,sample_from_group))
print(len(samples), num_to_select*len(ids))

39 39


In [53]:
# df_sample = df.loc[(df['ID'].isin(samples))]
for drug in drug_names:
    df_drug = df.loc[((df['DRUG'] == drug) | (df['DRUG'] == 'Controls'))&(df['YNAME'] == observation_name)]
    df_drug = df_drug.drop(df_drug[df_drug['OBS'] == '.'].index)
    df_drug['OBS'] = df_drug['OBS'].astype(float)
    df_sample = df_drug.loc[(df['ID'].isin(samples))]

    dose_unit = df_drug['UNIT'].values[0]
    
    x_label = "Time, hours"
    y_label = observation_name + ", " + dose_unit
    tick_distance = 48
    dose_categories = df_sample.DOSE.unique()
    
    fig = px.line(
        df_sample,
        title=drug + " Data: " + observation_name,
        x="TIME",
        y="OBS",
        line_group="ID",
        color="DOSE",
        width=800, 
        height=500,
        # category_orders = {"Dose":["0", "5", "10", "15"]},
        hover_data=list(df.columns))

    fig.update_xaxes(title_text=x_label, tick0=0, dtick=tick_distance)
    fig.update_yaxes(title_text=y_label)
    fig.update_traces(mode='markers+lines')
    fig.show()
    
    df_stats = df_drug[['TIME', 'DOSE', 'OBS']]
    df_stats = df_stats.groupby(["TIME", "DOSE"], as_index=False).filter(lambda x: len(x) > 1).groupby(["TIME", "DOSE"], as_index=False).agg({'OBS':['mean','std']})
    # print(df_stats)
    df_stats.columns = ['TIME', "DOSE", 'mean', 'std']
    # df_view = df_drug.loc[((24 <= df_drug['TIME'])& (df_drug['TIME']<= 72))]
    # print(df_view)

    fig = px.scatter(
        df_stats,
        title=observation_name + " Mean with" + drug,
        x="TIME",
        y="mean",
        error_y = "std",
        #line_group="ID",
        facet_col="DOSE", 
        # color="DOSE",
        width=1200, 
        height=400,
        # category_orders = {"Dose":["0", "5", "10", "15"]},
        # hover_data=list(df_stats.columns)
    )

    fig.update_xaxes(title_text=x_label, tick0=0, dtick=2*tick_distance)
    # fig.update_yaxes(title_text=y_label)
    fig.update_traces(mode='markers+lines')
    fig.show()

Next Notebook: [PK model - Naive Pooled Inference](PK_naive_pooled_inference.ipynb)